<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> <h1>LangChain for Generative AI</h1>
<h1>LangChain</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
from operator import itemgetter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import torch

import openai
from openai import OpenAI

import transformers
from transformers import pipeline
from transformers import set_seed
set_seed(42) # Set the seed to get reproducible results


import langchain
from langchain.chains import create_sql_query_chain
from langchain.tools import DuckDuckGoSearchRun

import langchain_openai
from langchain_openai import ChatOpenAI

import langchain_anthropic
from langchain_anthropic import ChatAnthropic


import langchain_core
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough

import langchain_community
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

import watermark

%load_ext watermark
%matplotlib inline

We start by print out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.12.3

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: cc526109102c100be511b85d0872c4bdfd674d94

pandas             : 2.1.4
langchain_anthropic: 0.1.15
transformers       : 4.41.1
torch              : 2.3.0
openai             : 1.30.5
langchain          : 0.2.2
langchain_core     : 0.2.3
langchain_community: 0.2.1
numpy              : 1.26.4
matplotlib         : 3.8.0
watermark          : 2.4.3
langchain_openai   : 0.1.8



Load default figure style

In [3]:
plt.style.use('./d4sci.mplstyle')

# OpenAI

The first step is generate API key on the OpenAI website and store it as the "OPENAI_API_KEY" variable in your local environment. Without it we won't be able to do anything. You can find your API key in your using settings: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key. Then we are ready to instantiate the client

In [5]:
client = OpenAI()

In [6]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
        {
            "role": "user", 
            "content": "What was Superman's weakness?"
        },
    ]
)

In [7]:
print(response)

ChatCompletion(id='chatcmpl-9Wq77ek9ml7zkQawe6QERj057RzFR', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Superman's most well-known weakness is Kryptonite, a mineral from his home planet of Krypton. There are several varieties of Kryptonite, but the most common, Green Kryptonite, can weaken Superman, drain his powers, and prolonged exposure could be fatal. Other notable weaknesses include:\n\n1. **Red Sun Radiation:** Superman derives his powers from Earth's yellow sun. Exposure to red sun radiation (similar to that of his home planet, Krypton) can strip him of his powers, making him as vulnerable as a regular human.\n   \n2. **Magic:** While not exactly a weakness, Superman is particularly vulnerable to magical attacks and powers. Unlike physical or energy-based attacks, magic can bypass his physical invulnerability.\n\n3. **Psionic Attacks:** Superman is also susceptible to mental attacks or manipulation, such as telepathy, thou

In [8]:
print(response.choices[0].message.content)

Superman's most well-known weakness is Kryptonite, a mineral from his home planet of Krypton. There are several varieties of Kryptonite, but the most common, Green Kryptonite, can weaken Superman, drain his powers, and prolonged exposure could be fatal. Other notable weaknesses include:

1. **Red Sun Radiation:** Superman derives his powers from Earth's yellow sun. Exposure to red sun radiation (similar to that of his home planet, Krypton) can strip him of his powers, making him as vulnerable as a regular human.
   
2. **Magic:** While not exactly a weakness, Superman is particularly vulnerable to magical attacks and powers. Unlike physical or energy-based attacks, magic can bypass his physical invulnerability.

3. **Psionic Attacks:** Superman is also susceptible to mental attacks or manipulation, such as telepathy, though he has shown some resistance to mind control at times.

These weaknesses add depth and complexity to his character, making his stories richer and more engaging.


# LangChain

We instantiate the LangChain interface for OpenAI

In [10]:
model = ChatOpenAI(model="gpt-4o")

In [11]:
messages = [
    SystemMessage(content="What was Superman's weakness?"),
]

model.invoke(messages)

AIMessage(content="Superman's most well-known weakness is Kryptonite, a radioactive substance from his home planet of Krypton. Exposure to Kryptonite can weaken Superman, strip him of his powers, and even potentially kill him if he is exposed to it for a prolonged period. There are different types of Kryptonite, each with varying effects:\n\n- **Green Kryptonite**: The most common form, which weakens and can potentially kill Superman with prolonged exposure.\n- **Red Kryptonite**: Causes unpredictable effects that vary with each exposure, such as altering his behavior or causing physical transformations.\n- **Gold Kryptonite**: Permanently removes Superman's powers if he is exposed to it.\n- **Blue Kryptonite**: Affects Bizarro, an imperfect clone of Superman, in the same way that Green Kryptonite affects Superman.\n\nAdditionally, Superman can be vulnerable to magic and magical attacks, which can bypass his usual invulnerability and cause him harm.", response_metadata={'token_usage': 

In [12]:
output = Out[11]

In [14]:
output.response_metadata["token_usage"]

{'completion_tokens': 188, 'prompt_tokens': 13, 'total_tokens': 201}

In [15]:
parser = StrOutputParser()

In [16]:
result = model.invoke(messages)

In [17]:
parser.invoke(result)

"Superman's most well-known weakness is Kryptonite, a radioactive mineral from his home planet of Krypton. When exposed to Kryptonite, Superman loses his powers and becomes vulnerable, and prolonged exposure can be fatal to him. There are different types of Kryptonite, each with varying effects:\n\n- **Green Kryptonite**: The most common form, which weakens Superman and can potentially kill him with prolonged exposure.\n- **Red Kryptonite**: Causes unpredictable and often temporary effects, such as physical transformations or emotional changes.\n- **Gold Kryptonite**: Can permanently remove Superman's powers.\n- **Blue Kryptonite**: Affects Bizarro, a flawed clone of Superman, in a similar way that Green Kryptonite affects Superman.\n- **Black Kryptonite**: Can split Superman into two separate beings, one good and one evil.\n\nIn addition to Kryptonite, Superman has other vulnerabilities:\n- **Red Sun Radiation**: Exposure to radiation from a red sun, like the one Krypton orbited, can 

Let us create our first chain. Stages of the chain are conencted with the pipe '|' character

In [18]:
chain = model | parser

Now whenver we call __invoke()__ on the chain, it automatically runs all the steps

In [19]:
chain.invoke(messages)

"Superman's primary weakness is Kryptonite, a radioactive mineral from his home planet of Krypton. Exposure to Kryptonite weakens him and can be fatal with prolonged exposure. There are different types of Kryptonite with varying effects, but the most common is green Kryptonite, which causes severe physical weakness and pain. Additionally, Superman is vulnerable to magic and can be harmed by magical spells and enchanted weapons. While he is invulnerable to most physical attacks, he can also be affected by extreme physical forces and beings of comparable or greater strength."

We can also create templates for our prompts, following conventions similar to the Jinja templating system

In [20]:
system_template = "Translate the following into {language}:"

And we can combine multiple messages into a single template

In [21]:
prompt_template = ChatPromptTemplate.from_messages(
    [
     ("system", system_template), 
     ("user", "{text}")]
)

To instantiate the prompt, we must provide the correct fields

In [22]:
result = prompt_template.invoke({
    "language": "italian", 
    "text": "Be the change that you wish to see in the world."
})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='Be the change that you wish to see in the world.')])

The full interaction is:

In [23]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='Be the change that you wish to see in the world.')]

In [25]:
chain = prompt_template | model | parser

In [26]:
chain.invoke({
    "language": "italian", 
    "text": "Be the change that you wish to see in the world."
})

'Sii il cambiamento che desideri vedere nel mondo.'

# Anthropic

In [27]:
model_a = ChatAnthropic(model="claude-3-opus-20240229")

In [28]:
chain_a = prompt_template | model_a | parser

In [29]:
chain_a.invoke({
    "language": "italian", 
    "text": "Be the change that you wish to see in the world."
})

'Sii il cambiamento che desideri vedere nel mondo.'

# Message History

In [30]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [31]:
config = {"configurable": {"session_id": "abc2"}}

In [32]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

'Hi Bob! How can I assist you today?'

In [33]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'You mentioned that your name is Bob. How can I help you today, Bob?'

In [34]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I don't have access to personal data about users unless it has been shared with me in the course of our conversation. Therefore, I don't know your name. How can I assist you today?"

In [35]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'You told me earlier that your name is Bob. Is there something specific you need help with today?'

In [36]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model | parser

In [37]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response

'Hi Bob! How can I assist you today?'

In [38]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [39]:
config = {"configurable": {"session_id": "abc5"}}

In [40]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response

'Hi Jim! How can I assist you today?'

In [41]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response

'Your name is Jim. How can I help you today, Jim?'

# Database Integration

In [42]:
db = SQLDatabase.from_uri("sqlite:///data/Northwind_small.sqlite")

In [43]:
print(db.dialect)

sqlite


In [44]:
print(db.get_usable_table_names())

['Category', 'Customer', 'CustomerCustomerDemo', 'CustomerDemographic', 'Employee', 'EmployeeTerritory', 'Order', 'OrderDetail', 'Product', 'Region', 'Shipper', 'Supplier', 'Territory']


In [45]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [46]:
write_query = create_sql_query_chain(llm, db)

In [47]:
response = write_query.invoke({"question": "How many customers are there"}) 
response

'SELECT COUNT("Id") AS TotalCustomers\nFROM Customer;'

In [48]:
db.run(response)

'[(91,)]'

In [49]:
execute_query = QuerySQLDataBaseTool(db=db)

In [50]:
sql_chain = write_query | execute_query

In [51]:
sql_chain.invoke({"question": "How many employees are there"})

'[(9,)]'

In [53]:
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

'There are a total of 9 employees.'

In [55]:
search = DuckDuckGoSearchRun()
search.run("When will the next solar eclipse be?")

"Find out when and where the next solar and lunar eclipses will occur. The next total solar eclipse will be on April 8, 2024, crossing North America. It will be 20 years before there's a chance to witness a total solar eclipse in the United States again. According to NASA, after Monday's total solar eclipse, the next one viewable from the ... Explore an interactive map of the paths and dates of the next 15 total solar eclipses, based on NASA's database of five millennium canon. Find out when and where you can see totality or partial eclipse in your region or around the world. On April 8, 2024, a total solar eclipse moved across North America, passing over Mexico, the United States, and Canada. A total solar eclipse happens when the Moon passes between the Sun and Earth, completely blocking the face of the Sun. The sky will darken as if it were dawn or dusk. Safety is the number one priority when viewing a total ... The next solar eclipse over the United States will be visible in Alaska

<center>
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>